# Kiến trúc light model (~1M tham số) cho CIFAR-10: MobileLite-C10

Mục tiêu là một mô hình nhỏ gọn, khoảng 1 triệu tham số, nhưng vẫn đủ mạnh để phân loại ảnh CIFAR-10 (32×32).  

## Ý tưởng chính
- Dùng **Depthwise Separable Convolution (DS-Conv)** để giảm tham số.  
- Khối cơ bản là **Inverted Residual Block** (còn gọi bottleneck).  
- Kích hoạt dùng **ReLU6** (giúp phân phối gọn hơn khi lượng tử).  
- Kết thúc bằng **Global Average Pooling (GAP)** thay vì fully-connected to.

## Cấu trúc MobileLite-C10
- **Stem**: Conv 3×3 (3→32α), stride=1, BN, ReLU6.  
- **Stage 1**: 1× bottleneck (t=4, k=3, stride=1, out=24).  
- **Stage 2**: 2× bottleneck (t=4, k=3, stride=2, out=32).  
- **Stage 3**: 2× bottleneck (t=4, k=3, stride=2, out=48).  
- **Stage 4**: 2× bottleneck (t=4, k=3, stride=1, out=64).  
- **Stage 5**: Conv 1×1 (64→128α), BN, ReLU6.  
- **Head**: GAP → FC (128α→10).  

Ở đây α là hệ số "width multiplier". Với α=0.75, mô hình khoảng 0.9–1.1M tham số.  

## Inverted Residual Block (bottleneck, t=4, k=3)
1. **Expand**: PW conv 1×1 mở rộng số kênh: $C_{mid} = t \cdot C_{in}$.  
2. **Depthwise conv**: DW conv 3×3 trên từng kênh (stride 1 hoặc 2).  
3. **Project**: PW conv 1×1 thu nhỏ về $C_{out}$.  
4. Nếu stride=1 và $C_{in}=C_{out}$ thì cộng residual.

Công thức block:

$$
Y =
\begin{cases}
X + \text{BN}(W_p * \phi(\text{BN}(W_d \odot \phi(\text{BN}(W_e * X))))) & \text{nếu stride=1, } C_{in}=C_{out} \\
\text{BN}(W_p * \phi(\text{BN}(W_d \odot \phi(\text{BN}(W_e * X))))) & \text{ngược lại}
\end{cases}
$$

Trong đó:
- $W_e$: kernel 1×1 (expand).  
- $W_d$: kernel depthwise 3×3.  
- $W_p$: kernel 1×1 (project).  
- $\phi$: ReLU6.  
- $\odot$: convolution từng kênh (depthwise).


# PTQ Phiên bản 1 — MinMax Calibration (chuẩn cơ bản)

## Ý tưởng
- Weights lượng tử **per-channel symmetric** (mỗi kênh có thang đo riêng, zero-point = 0).  
- Activations lượng tử **per-tensor asymmetric** (toàn tensor chung scale, có zero-point).  
- Calibration bằng min/max đơn giản, nhanh, dùng khoảng 512–1024 ảnh.

## Công thức lượng tử
- Quantize:  
  $$
  q = \text{clip}\left(\left\lfloor \frac{x}{s} \right\rceil + z,\ q_{min}, q_{max}\right)
  $$
- Dequantize:  
  $$
  \hat{x} = s \cdot (q - z)
  $$
- Với activations (asymmetric):  
  $$
  s = \frac{x_{max} - x_{min}}{q_{max} - q_{min}}, \quad
  z = \left\lfloor q_{min} - \frac{x_{min}}{s} \right\rceil
  $$
- Với weights (symmetric, per-channel):  
  $$
  s_c = \frac{\max(|x_{min,c}|, |x_{max,c}|)}{127}, \quad z_c = 0
  $$

## Trình tự
1. Train mô hình FP32 → lưu checkpoint.  
2. Fuse Conv–BN–ReLU.  
3. Gắn observer (weights per-channel, activations per-tensor).  
4. Chạy calibration (512–1024 ảnh không augment).  
5. Convert sang INT8.  
6. Đánh giá: accuracy, size, latency.  
7. Nếu accuracy drop nhiều, tăng ảnh calibration hoặc dùng clipping 99.9%.


# PTQ Phiên bản 2 — Histogram/KL Calibration + Bias Correction

## Ý tưởng
- MinMax dễ bị outlier. Thay vào đó, dùng histogram và chọn ngưỡng tối ưu bằng KL-divergence (hoặc MSE).  
- Sau lượng tử, áp dụng **bias correction** để bù lại sai số trung bình.

## Calibration bằng histogram + KL
- Xây histogram $H$ của activation.  
- Tìm ngưỡng $T$ để phân bố lượng tử $\tilde{P}_T$ gần với phân bố gốc $P$:  
  $$
  T^\star = \arg\min_T \mathrm{KL}(P(\cdot \mid |x| \le T)\,\|\,\tilde{P}_T)
  $$
- Sau đó scale: $s = T^\star / q_{max}$ (nếu symmetric).

## Bias correction
- Với lớp conv: $y = W * x + b$.  
- Sau lượng tử: $\hat{y} = \hat{W} * \hat{x} + \hat{b}$.  
- Ước lượng sai lệch:  
  $$
  \Delta_c = \mathbb{E}_{\mathcal{D}}[y_c - \hat{y}_c]
  $$
- Cập nhật bias:  
  $$
  \hat{b}_c \leftarrow \hat{b}_c + \Delta_c
  $$

## Trình tự
1. Train FP32, fuse Conv–BN–ReLU.  
2. Chèn observer histogram cho activations.  
3. Calibration (khoảng 1000 ảnh).  
4. Tìm ngưỡng tối ưu theo KL hoặc MSE.  
5. Convert sang INT8.  
6. Chạy bias correction bằng calibration set.  
7. Đánh giá lại.  
